# Join Adjacency Matrix, Gene Expression & Labels

In [1]:
import pandas
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import h5py
import mygene
%matplotlib inline

## Get Gene Expression

In [2]:
# params
TIMES_STD_THRESHOLD = 10.
MAX_ZEROS_ALLOWED = .5

In [3]:
# load data
gene_expression = pandas.DataFrame.from_csv('gene_expression/normalized-counts-labels.csv',
                                            encoding='utf-8', sep=',')
ge_nonames = gene_expression.drop('Name', axis=1)

# kick out super highly expressed genes
threshold = ge_nonames.mean(axis=1).std()*TIMES_STD_THRESHOLD
#print ("Threshold Gene Expression: {}".format(threshold))
anomalies = gene_expression[ge_nonames.mean(axis=1) > threshold]
ge_anomalies_removed = ge_nonames.drop(anomalies.index)

# kick out genes with too many zeros
ge_zeros_removed = ge_anomalies_removed[ge_anomalies_removed.astype('bool').mean(axis=1)>=(1-MAX_ZEROS_ALLOWED)]

# scaling
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(ge_zeros_removed)
ge_scaled = pandas.DataFrame(scaled_features,
                             index=ge_zeros_removed.index,
                             columns=ge_zeros_removed.columns)

# print some information
print ("Had gene expression for {} genes in the beginning.".format(ge_nonames.shape[0]))
print ("kicked out {} super highly expressed genes".format(ge_nonames.shape[0] - ge_anomalies_removed.shape[0]))
print ("Kicked out {} genes with more than {}% zeros".format(ge_anomalies_removed.shape[0]-ge_zeros_removed.shape[0],
                                                             MAX_ZEROS_ALLOWED*100.
                                                            ))
print ("==> Left with gene expression for {} genes".format(ge_scaled.shape[0]))

Had gene expression for 41424 genes in the beginning.
kicked out 9 super highly expressed genes
Kicked out 15021 genes with more than 50.0% zeros
==> Left with gene expression for 26394 genes


In [4]:
gene_names = gene_expression.loc[ge_scaled.index].Name.values
gene_ids = ge_scaled.index
gene_expression.head()

,Name,Pam3T16,Pam3T8,Pam3T16.1,Pam3T8.1,Pam3T16.2,Pam3T8.2,ControlT8,ControlT16,ControlT8.1,...,gfpmT8.1,gfpmT16.1,gfpmT8.2,gfpmT16.2,gfppT8,gfppT16,gfppT8.1,gfppT16.1,gfppT8.2,gfppT16.2
Ensembl-ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,TSPAN6,6.736327,8.519666,10.773706,10.813725,2.427830,7.425264,9.417385,7.920964,10.812953,...,12.529836,6.769370,13.440488,5.248289,14.583375,13.538040,18.220493,14.908853,16.156797,30.745221
ENSG00000000419,DPM1,97.676739,73.025712,132.362676,110.840686,87.401897,87.865624,92.725018,105.132797,121.218890,...,127.983328,147.421826,104.163784,86.596770,298.959192,239.172041,268.101547,155.611158,283.898001,247.298513
ENSG00000000457,SCYL3,84.204085,74.242807,61.564035,68.486928,78.499852,92.815800,86.929705,72.728852,66.585024,...,76.074006,63.180782,117.604272,94.469203,83.125239,79.724014,76.786365,62.430824,85.400212,93.572410
ENSG00000000460,C1orf112,61.468982,48.683808,70.798641,63.080065,63.123592,76.727728,68.819350,68.408327,77.397977,...,56.384263,72.958761,81.482960,51.608176,24.791738,33.092987,29.933668,59.635414,30.005480,26.734974
ENSG00000000938,FGR,53.890614,150.919804,143.136382,362.259802,156.999704,160.880721,62.299622,61.207450,189.511222,...,333.830638,257.988195,186.486774,70.851902,129.792040,97.774734,272.005939,245.996082,140.794944,101.592903


## ConsensusPathDB: Get PPI Network
I want to construct an adjacency matrix from the interactions in ConsensusPathDB.
However, I only want it for the genes for which I have gene expression data. If not both genes are involved in an interaction, I don't want to save the interaction.

In [93]:
# read interaction data (implicit threshold of 0.5 for interaction confidence)
interactions = pandas.read_csv('../data/networks/oneToOne-highConf-geneNames-CPDB.sif',
                               header=None,
                               sep='\(interacts\)',
                               encoding='utf8',
                               engine='python',
                               names=['gene1', 'gene2']
                              )

# construct Adjacency Matrix
N = len(gene_names)
print (N)
adjacency_matrix = np.zeros((N, N), np.uint8)
adj_df = pandas.DataFrame(adjacency_matrix, index=gene_names, columns=gene_names)

count = 0
gene_names_set = set(gene_names)
for index, row in interactions.iterrows():
    i1 = row.gene1.strip()
    i2 = row.gene2.strip()
    if i1 in gene_names_set and i2 in gene_names_set:
        adj_df.ix[i1, i2] = 1
        adj_df.ix[i2, i1] = 1
            
    count += 1
    if count % 10000 == 0:
        print ("Processed {} out of {} rows".format(count, interactions_nona.shape[0]))

26394
Processed 10000 out of 174225 rows
Processed 20000 out of 174225 rows
Processed 30000 out of 174225 rows
Processed 40000 out of 174225 rows
Processed 50000 out of 174225 rows
Processed 60000 out of 174225 rows
Processed 70000 out of 174225 rows
Processed 80000 out of 174225 rows
Processed 90000 out of 174225 rows
Processed 100000 out of 174225 rows
Processed 110000 out of 174225 rows
Processed 120000 out of 174225 rows
Processed 130000 out of 174225 rows
Processed 140000 out of 174225 rows
Processed 150000 out of 174225 rows
Processed 160000 out of 174225 rows
Processed 170000 out of 174225 rows


In [94]:
adj_df.sum().sum()

313102

## STRING DB: Get PPI network
I also want to construct the adjacency matrix for the STRING DB PPI network because it seems as if ConsensusPathDB is lacking some interactions I am interested in.

In [63]:
# load data
interactions = pandas.read_csv('../data/networks/9606_stringdb_v10_5.txt.gz',
                                         header=0,
                                         sep=' ',
                                         encoding='utf8'
                                        )
# remove NAs and the trailing species name
interactions_nona = interactions.dropna()
interactions_nona.protein1 = interactions_nona.protein1.str.replace('9606.', '')
interactions_nona.protein2 = interactions_nona.protein2.str.replace('9606.', '')

# plot distribution of scores
fig = plt.figure(figsize=(14, 8))
ax = interactions_nona.combined_score.plot.hist(bins=50, title='Distribution of Scores')
plt.xlabel('Combined Scores (STRING DB)')

KeyboardInterrupt: 

In [55]:
# use bioMart to convert protein and gene IDs
mg = mygene.MyGeneInfo()
f = lambda x: x['ensembl'][0]['gene'] if type(x['ensembl']) is list else x['ensembl']['gene']
res = mg.getgenes(interactions_nona.protein1, 'symbol,ensembl.gene', scopes='ensembl.protein')
proteins1 = [f(x) for x in res]
res = mg.getgenes(interactions_nona.protein2, 'symbol,ensembl.gene', scopes='ensembl.protein')
proteins2 = [f(x) for x in res]

# put into dataframe
interactions_nona['gene1'] = proteins1
interactions_nona['gene2'] = proteins2

querying 1-20...done.
querying 1-20...done.


In [56]:
interactions_nona

,protein1,protein2,combined_score,gene1,gene2
0,ENSP00000000233,ENSP00000263431,260,ENSG00000004059,ENSG00000126583
1,ENSP00000000233,ENSP00000353863,164,ENSG00000004059,ENSG00000163848
2,ENSP00000000233,ENSP00000342026,159,ENSG00000004059,ENSG00000117592
3,ENSP00000000233,ENSP00000240874,194,ENSG00000004059,ENSG00000160145
4,ENSP00000000233,ENSP00000379847,164,ENSG00000004059,ENSG00000166478
5,ENSP00000000233,ENSP00000400088,189,ENSG00000004059,ENSG00000250506
6,ENSP00000000233,ENSP00000362465,240,ENSG00000004059,ENSG00000102383
7,ENSP00000000233,ENSP00000415070,164,ENSG00000004059,ENSG00000270011
8,ENSP00000000233,ENSP00000379387,164,ENSG00000004059,ENSG00000283034
9,ENSP00000000233,ENSP00000335632,224,ENSG00000004059,ENSG00000187446


In [62]:
INTERACTION_SCORE_THRESHOLD = 1

# extract adjacency matrix from interactions
N = len(gene_names)
adjacency_matrix = np.zeros((N, N), np.uint8)
adj_df = pandas.DataFrame(adjacency_matrix, index=gene_ids, columns=gene_ids)

count = 0
for index, row in interactions_nona.iterrows():
    if row.combined_score > INTERACTION_SCORE_THRESHOLD:
        i1 = row.gene1
        i2 = row.gene2
        if i1 in gene_ids and i2 in gene_ids:
            adj_df.ix[i1, i2] = 1
            adj_df.ix[i2, i1] = 1
        else:
            print ("not found {} -> {}".format(i1, i2))
            
    count += 1
    if count % 10000 == 0:
        print ("Processed {} out of {} rows".format(count, interactions_nona.shape[0]))

not found ENSG00000004059 -> ENSG00000126583
not found ENSG00000004059 -> ENSG00000283034


In [61]:
test = adj_df.sum()
test[test >= 1]

Ensembl-ID
ENSG00000004059    18
ENSG00000037042     1
ENSG00000102383     1
ENSG00000107104     1
ENSG00000117592     1
ENSG00000141736     1
ENSG00000154710     1
ENSG00000160145     1
ENSG00000163848     1
ENSG00000166478     1
ENSG00000167981     1
ENSG00000170382     1
ENSG00000171425     1
ENSG00000182704     1
ENSG00000187446     1
ENSG00000189045     1
ENSG00000196372     1
ENSG00000250506     1
ENSG00000270011     1
dtype: int64

## Remove isolated genes from network & gene expression

In [95]:
# kick out all genes without interactions (isolated nodes)
genes_with_interactions = adj_df.sum(axis=1) > 0
ppi_network = adj_df.ix[genes_with_interactions, genes_with_interactions]

# remove gene expression for isolated genes
gene_names = ppi_network.index
gene_expression_short = ge_scaled.loc[gene_expression['Name'].isin(gene_names)]
# build list of gene names and ids
gene_names_ids = np.stack([np.array(gene_expression_short.index),
                           np.array(gene_names)],
                          axis=1
                         )


,Pam3T16,Pam3T8,Pam3T16.1,Pam3T8.1,Pam3T16.2,Pam3T8.2,ControlT8,ControlT16,ControlT8.1,ControlT16.1,...,gfpmT8.1,gfpmT16.1,gfpmT8.2,gfpmT16.2,gfppT8,gfppT16,gfppT8.1,gfppT16.1,gfppT8.2,gfppT16.2
Ensembl-ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,-0.112112,-0.116104,-0.066323,-0.129456,-0.128630,-0.085007,-0.119097,-0.132638,-0.137808,-0.116562,...,-0.124763,-0.137221,-0.091480,-0.122248,-0.071886,-0.076810,-0.049512,-0.101443,-0.071697,-0.067789
ENSG00000000419,-0.019636,-0.055894,-0.014149,-0.026348,-0.034298,-0.031775,-0.027954,-0.010898,0.011211,0.011268,...,-0.003864,0.022521,-0.018581,-0.039281,0.048819,0.030559,0.013568,0.014119,0.039539,0.028876
ENSG00000000457,-0.033336,-0.054758,-0.044529,-0.070006,-0.044180,-0.028499,-0.034294,-0.051478,-0.062530,-0.056726,...,-0.058222,-0.073153,-0.007781,-0.031252,-0.042793,-0.045315,-0.034728,-0.062412,-0.042929,-0.039745
ENSG00000000938,-0.064162,0.016814,-0.009526,0.232816,0.042965,0.016543,-0.061241,-0.065906,0.103387,-0.027715,...,0.211692,0.148093,0.047569,-0.055339,-0.022985,-0.036726,0.014554,0.088354,-0.019915,-0.036164
ENSG00000000971,-0.100125,-0.103608,-0.052454,-0.085798,-0.098084,-0.074361,-0.124645,-0.138950,-0.131663,-0.112935,...,-0.125700,-0.125262,-0.096880,-0.120464,-0.075600,-0.079674,-0.052798,-0.104504,-0.078409,-0.080917


## Get Labels

In [98]:
ppi_network.shape

(10367, 10367)

## Generate Training, Testing & Validation Splits

## Write to hdf5 File on Disk

In [99]:
# create gene names
string_dt = h5py.special_dtype(vlen=str)

f = h5py.File('../data/preprocessing/ppi_networks.h5', 'w')
f.create_dataset('consensusPathDB_ppi', data=ppi_network, shape=ppi_network.shape)
f.create_dataset('gene_expression', data=gene_expression_short, shape=gene_expression_short.shape)
f.create_dataset('gene_names', data=gene_names_ids, dtype=string_dt)

# doesn't exist yet :-(
#f.create_dataset('y_train', data=y_train, shape=y_train.shape)
#f.create_dataset('y_test', data=y_test, shape=y_test.shape)
#f.create_dataset('y_val', data=y_val, shape=y_val.shape)

#f.create_dataset('mask_train', data=mask_train, shape=mask_train.shape)
#f.create_dataset('mask_test', data=mask_test, shape=mask_test.shape)
#f.create_dataset('mask_val', data=mask_val, shape=mask_val.shape)
f.close()

In [25]:
f.close()

In [3]:
fname = '../data/preprocessing/ppi_networks.h5'
with h5py.File(fname, 'r') as f:
    gene_expression_data = f['gene_expression'][:]
    ppi_network = f['consensusPathDB_ppi'][:]
    gene_names = f['gene_names'][:]

In [4]:
gene_names

array(['DPM1', 'FGR', 'NFYA', ..., 'MRM1', 'DGCR6', 'MATR3'], dtype=object)

In [121]:
pr_val = np.array([3, 1, 2])
gene_names = np.array(['g1', 'g2', 'g3'])
sort_idx = pr_val.argsort()
gene_names_sorted = gene_names[sort_idx[::-1]]

In [122]:
gene_names_sorted

array(['g1', 'g3', 'g2'],
      dtype='<U2')

In [124]:
import networkx as nx

In [125]:
G = nx.DiGraph(nx.path_graph(4))
pr = nx.pagerank(G)

In [126]:
pr

{0: 0.17543839772251535,
 1: 0.32456160227748465,
 2: 0.32456160227748465,
 3: 0.17543839772251535}

In [128]:
import operator
pr_s = sorted(pr.items(), key=operator.itemgetter(1))[::-1]

In [131]:
gene_names_both

array([['TSPAN6', 'ENSG00000000003'],
       ['DPM1', 'ENSG00000000419'],
       ['SCYL3', 'ENSG00000000457'],
       ..., 
       ['CH17-132F21.5', 'ENSG00000281904'],
       ['RP11-439M15.1', 'ENSG00000281909'],
       ['LINC01144', 'ENSG00000281912']], dtype=object)

In [135]:
count = 1
for gene_idx, pr in pr_s:
    print ("{}\t{}\t{}\t{}\n".format(gene_names_both[gene_idx][1], gene_names_both[gene_idx][0], count, pr))
    count += 1

ENSG00000000457	SCYL3	1	0.32456160227748465

ENSG00000000419	DPM1	2	0.32456160227748465

ENSG00000000460	C1orf112	3	0.17543839772251535

ENSG00000000003	TSPAN6	4	0.17543839772251535



In [12]:
import tables

ModuleNotFoundError: No module named 'tables'